In [1]:
from jinja2 import Environment, FileSystemLoader
import numpy as np
import pandas as pd 

In [2]:
opt_f='bromide.log'
not_opt_f='coumarin.log'
with open(not_opt_f, 'r') as f:
    lines = f.readlines()

In [3]:
# Check if optimization was succesfull 
opt = False
for line in lines:
    if  "**** THE GEOMETRY SEARCH IS NOT CONVERGED! ****" in line:
        opt = True

In [4]:
# If optimization is successfull 
if not opt:
    data = [] 
    for num, line in enumerate(lines):
        if "TOTAL NUMBER OF ATOMS" in line:
            n_atoms = int(line.split()[5])
        if "EQUILIBRIUM" in line:
            start_coord_index = num + 4
            end_coord_index = start_coord_index + n_atoms
            for i in range(start_coord_index, end_coord_index):
                data.append(lines[i].split())
    atomic_number = []
    element_symbol = []
    x_coord = []
    y_coord = []
    z_coord = []
    for line in data:
        element_symbol.append(line[0])
        atomic_number.append(line[1])
        x_coord.append(line[2])
        y_coord.append(line[3])
        z_coord.append(line[4])
        
    df = pd.DataFrame([element_symbol, x_coord, y_coord, z_coord])
    df = df.transpose()
    df[0] = df[0].map("{0:<4s}".format)
    df[1] = pd.to_numeric(df[1], downcast="float").map("{0:>16.10f}".format)
    df[2] = pd.to_numeric(df[2], downcast="float").map("{0:>16.10f}".format)
    df[3] = pd.to_numeric(df[3], downcast="float").map("{0:>16.10f}".format)
    
    result = df[0] + df[1] + df[2] + df[3]

In [5]:
file_loader=FileSystemLoader("../templates")
environment = Environment(loader=file_loader)
if not opt:
    template = environment.get_template("structure_input_template.xyz")

    MOLNAME="bromide"
    content = template.render(N_ATOMS=n_atoms, MOLNAME=MOLNAME, COORDINATES=result)
    print(content)
    
    # write into a file
    # filename = MOLNAME+'.xyz'
    # with open(filename, mode="w", encoding="utf-8") as message:
    #    message.write(content)

In [6]:
# If optimization is NOT successfull 

if  opt:
	template = environment.get_template("gamess_input_template.inp")
	data = []
	for num, line in enumerate(lines):
		if "TOTAL NUMBER OF ATOMS" in line:
			n_atoms = int(line.split()[5])
		if "ALWAYS THE LAST POINT COMPUTED!" in line:
			start_coord_index = num + 4
			end_coord_index = start_coord_index + n_atoms
			for i in range(start_coord_index, end_coord_index):
				data.append(lines[i])
				
	with open("gamess_parameters.in", 'r') as f:
		lines = f.readlines()
	for line in lines:
		if "SCFTYP" in line:
			SCFTYP = line.split()[2]
		if "MULT" in line:
			MULT = line.split()[2]
		if "OPTTOL" in line:
			OPTTOL = line.split()[2]
		if "NSTEP" in line:
			NSTEP = line.split()[2]
		if "GBASIS" in line:
			GBASIS = line.split()[2]
		if "NGAUSS" in line:
			NGAUSS = line.split()[2]
		if "MOLNAME" in line:
			MOLNAME = line.split()[2]


	content = template.render(SCFTYP=SCFTYP, MULT=MULT, OPTTOL=OPTTOL,
			NSTEP=NSTEP, GBASIS=GBASIS, NGAUSS=NGAUSS, MOLNAME=MOLNAME, COORDINATES=data)
	print(content)
	
	# write into a file
	# filename = MOLNAME+'.inp'
	# with open(filename, mode="w", encoding="utf-8") as message:
	# 	message.write(content)

 $CONTRL SCFTYP=RHF MULT=1 RUNTYP=OPTIMIZE $END
 $STATPT OPTTOL=0.0001 NSTEP=100 $END
 $BASIS  GBASIS=N31 NGAUSS=6 $END

$DATA
C1
 O           8.0  -3.1653408462   0.6356877423  -0.0159611523
 C           6.0  -2.0638705169   0.1433353987  -0.0422337969
 O           8.0  -0.9641934909   0.9716032877  -0.0674019256
 C           6.0  -1.7750007318  -1.2871835302  -0.0521999738
 C           6.0   0.3390418371   0.5322400519  -0.0396322582
 C           6.0  -0.5182488792  -1.7406343568  -0.0355879939
 C           6.0   0.6118633369  -0.8297790609  -0.0099302446
 C           6.0   1.3447358463   1.4824675604  -0.0359253715
 C           6.0   1.9458791868  -1.2400669489   0.0462841369
 C           6.0   2.6580408122   1.0544885021   0.0182395050
 C           6.0   2.9623278007  -0.3066132702   0.0659246829
 H           1.0  -2.6260430679  -1.9339135387  -0.0683227935
 H           1.0  -0.3226127769  -2.7958011481  -0.0418992733
 H           1.0   1.0829097033   2.5196494116  -0.0709490648
 H

In [7]:
with open("cluster_parameters.in", 'r') as f:
    lines = f.readlines()
for line in lines:
    if "WALLTIME" in line:
        WALLTIME = line.split()[2]
    if "BIN" in line:
        BIN = line.split()[2]
    if "USER_EMAIL" in line:
        USER_EMAIL = line.split()[2]

template = environment.get_template("array_sub_template.sh")

content = template.render(WALLTIME=WALLTIME, BIN=BIN, USER_EMAIL=USER_EMAIL)
print(content)

#!/bin/sh
file_xyz=`basename $(ls *.xyz)`
file_in=`basename $(ls *.in)`

keyword_ntraj=`head -n 1 $file_in | awk '{ print $1 }'` 

if [[ "$keyword_ntraj" ]]; then
    n_traj=`head -n 1 $file_in | awk '{ print $2 }'`
else 
    n_traj=$((`head -n 1 $file_xyz`*25))
fi

molname=`basename $file_xyz .xyz`
walltime=02:00:00
bin=/storage/brno2/home/wrojasv/MassSpec/QC_Spectra_Predictions/local_run_test/bin_test
user_email=wrojasv@recetox.muni.cz 

echo 'submitted neutral MD job'
job1=$(qsub -N $molname -M $user_email -l walltime=$walltime -v "bin=$bin" $bin/neutral_run_md.pbs)

echo 'submitted prep production MD job'
job2=$(qsub -W depend=afterok:$job1 -N $molname -M $user_email -l walltime=$walltime -v "bin=$bin" $bin/prep_prod_run_ei_md.pbs)

echo 'submitted production run EI MD job'
job3=$(qsub -W depend=afterok:$job2 -N $molname -M $user_email -l walltime=$walltime -v "bin=$bin" -J 1-$n_traj $bin/prod_run_ei_md.pbs)
